<a href="https://colab.research.google.com/github/domsoria/FaceEmotion/blob/master/RNN_ICR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RNN-ICR
## ***textgenrnn*-based Italian Cooking Recipe generator**
by Lorenzo Callegari

Trained on dataset scraped from [GialloZafferano.it](https://www.giallozafferano.it/)  
For more infromation about textgenrnn, you can visit [this GitHub repository](https://github.com/minimaxir/textgenrnn).  


In [ ]:
!pip install -q textgenrnn
from google.colab import files
from textgenrnn import textgenrnn
from datetime import datetime
import os


model_cfg = {
    'word_level': False,   # set to True if want to train a word-level model (requires more data and smaller max_length)
    'rnn_size': 128,   # number of LSTM cells of each layer (128/256 recommended)
    'rnn_layers': 3,   # number of LSTM layers (>=2 recommended)
    'rnn_bidirectional': False,   # consider text both forwards and backward, can give a training boost
    'max_length': 30,   # number of tokens to consider before predicting the next (20-40 for characters, 5-10 for words recommended)
    'max_words': 10000,   # maximum number of words to model; the rest will be ignored (word-level model only)
}

train_cfg = {
    'line_delimited': False,   # set to True if each text has its own line in the source file
    'num_epochs': 20,   # set higher to train the model for longer
    'gen_epochs': 5,   # generates sample text from model after given number of epochs
    'train_size': 0.8,   # proportion of input data to train on: setting < 1.0 limits model from learning perfectly
    'dropout': 0.0,   # ignore a random proportion of source tokens each epoch, allowing model to generalize better
    'validation': False,   # If train__size < 1.0, test on holdout dataset; will make overall training slower
    'is_csv': False   # set to True if file is a CSV exported from Excel/BigQuery/pandas
}

file_name = "gz_dataset.txt"
model_name = 'CNN_ICR'   # change to set file name of resulting trained models/texts

Here training starts! It's going to take a while..

In [ ]:
textgen = textgenrnn(name=model_name)

train_function = textgen.train_from_file if train_cfg['line_delimited'] else textgen.train_from_largetext_file

train_function(
    file_path=file_name,
    new_model=True,
    num_epochs=train_cfg['num_epochs'],
    gen_epochs=train_cfg['gen_epochs'],
    batch_size=1024,
    train_size=train_cfg['train_size'],
    dropout=train_cfg['dropout'],
    validation=train_cfg['validation'],
    is_csv=train_cfg['is_csv'],
    rnn_layers=model_cfg['rnn_layers'],
    rnn_size=model_cfg['rnn_size'],
    rnn_bidirectional=model_cfg['rnn_bidirectional'],
    max_length=model_cfg['max_length'],
    dim_embeddings=100,
    word_level=model_cfg['word_level'])

AttributeError: ignored

You can **generate a new recipe** using the cell below!  
Feel free to re-run the cell as many times as you want until you're satisfied with the recipe!

In [ ]:
# this temperature schedule cycles between 1 very unexpected token, 1 unexpected token, 2 expected tokens, repeat.
# changing the temperature schedule can result in wildly different output!
temperature = [1.0, 0.5, 0.2, 0.2]   
prefix = None   # if you want each generated text to start with a given seed text

if train_cfg['line_delimited']:
  n = 1000
  max_gen_length = 60 if model_cfg['word_level'] else 300
else:
  n = 1
  max_gen_length = 2000 if model_cfg['word_level'] else 10000

textgen.generate(temperature=temperature,
                 prefix=prefix,
                 n=n,
                 max_gen_length=max_gen_length)

NameError: ignored

If you'd like you can download the weights and configuration files using the cell below, allowing you to easily recreate the model on your own computer

In [ ]:
files.download(f'{model_name}_weights.hdf5')
files.download(f'{model_name}_vocab.json')
files.download(f'{model_name}_config.json')

To run the model on your pc simply create a Python script with the following code:


```
from textgenrnn import textgenrnn
textgen = textgenrnn(weights_path='RNN-ICR_weights.hdf5',
                       vocab_path='RNN-ICR_vocab.json',
                       config_path='RNN-ICR_config.json')

textgen.generate_samples(max_gen_length=1000)
textgen.generate_to_file('RNN-ICR_gentext.txt', max_gen_length=1000)
```

